In [ ]:
import pandas as pd
import autograd.numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from autograd import grad, jacobian, hessian

# Read in Data

In [ ]:
# Read a formatted csv with [A],[B],Ra,Rd,constant,constant_value 
# Constants needed are "hbd_yo", "ekt", and "Ksp"
def get_data_from_csv(filename):
    df = pd.read_csv(filename) 
    A = df['A'].to_numpy(dtype='float64')
    B = df['B'].to_numpy(dtype='float64')
    R_a = df['Ra'].to_numpy(dtype='float64')
    R_d = df['Rd'].to_numpy(dtype='float64')
    constants = {}
    for i in range(len(df['constant'])):
        if not pd.isna(df['constant'][i]):
            constants[df['constant'][i]] = df['constant_value'][i]
    return A, B, R_a, R_d, constants

# Equations

The function `find_var` computes rho, Pa, and Pb which are used by `find_R_a` and `find_R_b` to find the attachment and detachment rates. 

For given inputs, $[A],[B],v_a,v_b,k_a,k_b$ with constants $K_{sp}, \epsilon/kt$

$S = \sqrt{\frac{[A][B]}{K_{sp}}}$

$i_{super,A} = 2 * e^{-2\epsilon / kt} (S^2 - 1)\frac{v_bk_a[A]}{k_a[A] + v_b}$

$i_{super,B} = 2 * e^{-2\epsilon/kt}(S^2 - 1)\frac{v_ak_b[B]}{k_b[B]+v_a}$

$i_{super} = \frac{i_{super,A} + i_{super,B}}{2}$

$i_{under} = \Big(\frac{v_av_be^{-2\epsilon/kt}}{(2-S^2)v_b + k_a[A]} + \frac{v_av_be^{-2\epsilon/kt}}{(2-S^2)v_a + k_b [B]}\Big)(1-S^2)$

$i = \max(i_{super},i_{under})$

$u = \Big|2\frac{k_ak_b[A][B] - v_av_b}{k_a[A] + k_b[B] + v_a + v_b}\Big|$

$\rho = \sqrt{\frac{2i}{u}}$

$P_a = \frac{k_a[A] + v_b}{k_a[A] + k_b[B] + v_a + v_b}$

$P_b = \frac{k_b[B] + v_a}{k_a[A] + k_b[B] + v_a + v_b}$

`find_var` returns the values for $\rho, P_a, P_b$

Then you can compute $R_a, R_d$ using these formulas

$R_a  = \rho \cdot k_a [A] P_b (\frac{hbd}{y_0})$

$R_d = \rho \cdot v_a P_a (\frac{hbd}{y_0})$




In [ ]:
# Equations
def find_var(v_a, v_b, k_a, k_b, A, B, constants):
    Ksp = constants["Ksp"]
    ekt = constants["ekt"]

    S = np.sqrt(A * B / Ksp)

    i_A = 2 * np.exp(-2 * ekt) * (S**2 - 1) * (v_b * k_a * A)/(k_a * A + v_b)
    i_B = 2 * np.exp(-2 * ekt) * (S**2 - 1) * (v_a * k_b * B) / (k_b * B + v_a)
    i_super = (i_A + i_B) / 2
    i_under = ((v_a * v_b * np.exp(-2 * ekt))/((2 - S**2) * v_b + k_a * A) + ((v_a * v_b * np.exp(-2 * ekt)) / ((2 - S**2) * v_a + k_b * B))) * (1 - S**2)

    i = np.maximum(i_super,i_under)

    u = np.abs(2 *(k_a * k_b * A * B - v_a * v_b) / (k_a * A + k_b * B + v_a + v_b))

    rho = np.sqrt(2 * i / u)

    P_a = (k_a * A + v_b) / (k_a * A + k_b * B + v_a + v_b)
    P_b = (k_b * B + v_a) / (k_a * A + k_b * B + v_a + v_b) 

    return rho, P_a, P_b

def find_R_a(v_a, v_b, k_a, k_b, A, B, constants):
    hbd_yo = constants["hbd_yo"]
    rho, P_a, P_b = find_var(v_a, v_b, k_a, k_b, A, B, constants)

    R_a = rho * k_a * A * P_b * hbd_yo
    return R_a * 10**9

def find_R_d(v_a, v_b, k_a, k_b, A, B, constants):
    hbd_yo = constants["hbd_yo"]
    rho, P_a, P_b = find_var(v_a, v_b, k_a, k_b, A, B, constants)

    R_d = rho * v_a * P_a * hbd_yo
    return R_d * 10**9

Calculate the residuals. Given experimental data arrays $R^{exp}_a, R^{exp}_d$ of size n x 1

we calculate the loss function as 

$\mathcal{L}(v_a,v_b,k_a,k_b,constants) = \sum_{i = 1}^{n} (R^{exp}_{a,i} - R^{fit}_{a,i})^2 + \sum_{j = 1}^n (R^{exp}_{d,j} - R^{fit}_{d,j})^2$

This is the residual sum square of all the data.

In [ ]:
def residual_R_a(x, A, B, R_d, R_a, constants):
    return find_R_a(*x, A, B,constants) - R_a

def residual_R_d(x, A, B, R_d, R_a, constants):
    return find_R_d(*x,A,B, constants) - R_d

def residual_R(x, A, B, R_d, R_a, constants):
    return (find_R_d(*x,A,B, constants) - R_d) + (find_R_a(*x, A, B, constants) - R_a)

def minimize_R(x, A, B, R_d, R_a, constants):
    return np.sum(np.square(residual_R(x, A, B, R_d, R_a, constants)))

# Plot Function

In [ ]:
# Plotting
def plot_flux(A, B, R_d, R_a, constants, parameters):
    x = A / B
    x_sort = x.argsort()

    plt.plot(x, R_d,'o',label="J+")
    plt.plot(x[x_sort], find_R_d(*parameters,A[x_sort], B[x_sort], constants),'o',label="min_fit")
    plt.ylabel('J')
    plt.xlabel('[A] / [B]')
    plt.xscale('log')
    plt.legend()
    plt.show()

    plt.plot(x, R_a, 'o',label="J-")
    plt.plot(x[x_sort], find_R_a(*parameters,A[x_sort], B[x_sort], constants),'o',label="fit")
    plt.ylabel('J')
    plt.xlabel('[A] / [B]')
    plt.xscale('log')
    plt.legend()
    plt.show()

# Begin Program

In [ ]:
#%% Begin Program

# Filename
file = ""

# Get Data
A, B, R_d, R_a, constants = get_data_from_csv(file)

# Set Initial Value (Based on experimentation, we've determine that this is a good initial guess)
x0 = np.array([1.88593864e+01, 2.04204732e+01, 5.39999999e+05, 7.45500000e+06])

# Optimize
# We use 'trust-ncg' or trust-region Newton conjugate gradient method since of the optimization algorithms in 
# scipy this seemed to be the most robust. You can change the "method" parameter to use different optimizations.
# to learn more see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
min_lsq = minimize(minimize_R, x0, jac=jacobian(minimize_R), hess=hessian(minimize_R),args=(A,B,R_d,R_a,constants),method='trust-ncg', options={'maxiter':5000})
print(min_lsq)

# Plot
plot_flux(A, B, R_d, R_a, constants, min_lsq.x)

#%% See how much progress with residual sum squared
print("Initial R_a: ",np.sum(np.square(residual_R_a(x0, A, B, R_d, R_a, constants))))
print("Initial R_d: ",np.sum(np.square(residual_R_d(x0, A, B, R_d, R_a, constants))))
print("Fitted R_a: ",np.sum(np.square(residual_R_a(min_lsq.x, A, B, R_d, R_a, constants))))
print("Fitted R_d: ",np.sum(np.square(residual_R_d(min_lsq.x, A, B, R_d, R_a, constants))))
